# Preparation of data from Aspen HYSYS

This notebook is used to import data from raw standard reports from Aspen HYSYS, and transforms it in better structures information to handle analyses of the system.

In [1]:
import pandas as pd
import numpy as np
import math

### **CHANGE THE FILE ADDRESSES BELOW TO YOUR FILES' LOCATION AND NAMES!**

In [2]:
folder_path = 'C:\\Users\Italo\Google Drive\Dissertação Mestrado\Simulações\Procedimento de análise\\' #folders where the files are

materials_file='materials.txt' #file for the material-streams report
#Create Aspen HYSYS Report following Home>Reports>Insert(or Edit) Datasheet: Select "Material streams" and only "Attachments". Remember to select
#"pick all objects of the same type"


energy_file= 'energy.txt' #file for the energy-streams report
#Create Aspen HYSYS Report following Home>Reports>Insert (or Edit) Datasheet: select "Energy streams" and only "Attachments"


h0s0_file='h0-s0.txt' #file for the "user-variables" report from HYSYS
#this uses User-variables defined in Aspen HYSYS with VBA code, that specify the variables h0 (enthalpy at p0 and T0) and s0 (entropy at p0 and T0)
# Create Aspen HYSYS report following Home>Reports>Insert (or edit) Datasheet: select "material streams" and only "User variables"


hysys_report="relatório hysys.xlsx" #file with the "hysys report" standard HYSYS excel report
#Create the report "hysys" in "Excel reports" by following Home>Reports in Aspen HYSYS. This brings information to an excel file
# separated in different sheets for each type of equipment. It also brings the value of every energy stream related to their names.


streams_file= "SCOC-CC-data-import.xlsm" #excel file with the data importing VBA code from Aspen

In [3]:
#dictionary relating the desired variables and their rows in the VBA report output (variable names are in portuguese here)
dict_keys= ['Corrente','Fração de vapor','Temperatura (C)','Pressão (kPa)','Vazão molar (kmol/s)','Vazão mássica (kg/s)',\
            'Massa específica (kg/m³)','Entalpia molar (kJ/kmol)','Entropia molar (kJ/kmolK)','Comp. C1', 'Comp. C2', 'Comp. C3',\
            'Comp. C4','Comp. C5', 'Comp. CO2','Comp. N2','Comp. O2', 'Comp. H2O','Comp. Ar','Comp. TEG','Comp. H2S', 'LHV (kJ/kmol)']
dict_rows=[185,179,159,145,136,132,143,144,58,59,60,66,67,61,62,63,64,65,69,68,127] #these rows should be reviewed because the number of substances in the
#simulation change the disposition in the report's output
#there is no row corresponding to "Corrente", which is the stream's name, in dict_rows. This will be handled later.

#Some overall parameters for later analysis:
efic_mec=0.995 #mechanical efficiency of electric generator
efic_transf=0.997 #transformer efficiency to high-voltage for distribution
oxy_cost= 1026 #energy cost in kJ/kg O2 at 97% purity (O2 rich current delivered at 600kPa)

## For the exergy analysis, define T0, p0, and reference environment composition

In [4]:
T0=25+273.15
p0=101.325
phi0=60/100
p_sat= 3.169 #pressão de saturação da água a T0
R=8.314


In [5]:
#Defining the reference environment composition

#SZARGUT, J. and PÉTELA, R., Egzergia (in Polish), Wydawnictwa NaukowoTechniczne, Warsaw, 1965
#SZARGUT, J., Thermodynamic and Economic Analysis in Power Generation Industry, (in Polish), Wydawnictwa Naukowo—Techniczne, Warsaw, 1983. 
#Szargut, J., Exergy Method. Technical and Ecological Applications, WIT Press, Southampton, UK, 2005
# exergias químicas de Kotas (1985) (reproduzido de Szargut) em kJ/kmol, ref 298,15K, 101,325 kPa. (valor de TEG na verdade é etileno-glicol)
#valores nas variáveis p/H20 g. Para H2O líquido: 18,015, -44030, 3120

substancias=['N2','H2O','Ar','CO2','O2','C1','C2','C3','C4','C5','TEG','H2S']
frac_mol=[0.7803,0,0.00933,0.0003,0.2099,0,0,0,0,0,0,0]
massa_molar=[28.013,18.015,39.948,44.01,32,16.042,30.068,44.094,58.120,72.146,62.068,34.080]
h_d_std=[0,0,0,0,0,802320,1428780,2045380,2658830,3274290,1220530,946420]
b_ch_std=[720,11710 ,11690,20140 ,3970 ,836510 ,1504360,2163190 ,2818930 ,3477050 ,1214210,804770]

comp_amb=pd.DataFrame({'Substância':substancias,'Frac. molar (dry air)':frac_mol,'Massa molar (kg/kmol)':massa_molar, \
                      'Enthalpy of devaluation (kJ/kg)':h_d_std,'Standard chemical exergy (kJ/kmol)':b_ch_std})

p_agua=p_sat*phi0
#p_agua = 0.88   

comp_amb['p parcial (ar úmido)']=comp_amb['Frac. molar (dry air)']*comp_amb['Frac. molar (dry air)'].sum()*(p0-p_agua)
comp_amb.loc[1,'p parcial (ar úmido)']=p_agua
comp_amb['-T0*R*ln(pi/p0)']=-R*T0*np.log((comp_amb['p parcial (ar úmido)']/p0)) #this column corresponds to the standard chemical exergy
#if there are differences they are probably due to details in the choice of the reference environment composition


comp_amb.loc[5:,'-T0*R*ln(pi/p0)']=0
comp_amb

C:\Users\Italo\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Substância,Frac. molar (dry air),Massa molar (kg/kmol),Enthalpy of devaluation (kJ/kg),Standard chemical exergy (kJ/kmol),p parcial (ar úmido),-T0*R*ln(pi/p0)
0,N2,0.78030,28.013,0,720,77.567046,662.316902
1,H2O,0.00000,18.015,0,11710,1.901400,9855.146980
2,Ar,0.00933,39.948,0,11690,0.927464,11634.669462
3,CO2,0.00030,44.010,0,20140,0.029822,20154.885854
4,O2,0.20990,32.000,0,3970,20.865466,3917.123465
5,C1,0.00000,16.042,802320,836510,0.000000,0.000000
6,C2,0.00000,30.068,1428780,1504360,0.000000,0.000000
7,C3,0.00000,44.094,2045380,2163190,0.000000,0.000000
8,C4,0.00000,58.120,2658830,2818930,0.000000,0.000000
9,C5,0.00000,72.146,3274290,3477050,0.000000,0.000000


### Importing the data

In [6]:
energy = pd.read_table(folder_path+energy_file, header=None)
materials = pd.read_table(folder_path+materials_file, delim_whitespace=False,header=None)
equipamentos=pd.DataFrame(columns=['Equipamento','Tipo','Duty in','Duty out','Inlet 1','Inlet 2','Inlet 3','Inlet 4','Inlet 5','Inlet 6','Inlet 7',\
                                   'Outlet 1','Outlet 2','Outlet 3','Outlet 4','Outlet 5','Outlet 6','Outlet 7'])


#this loop designates the names of the energy streams to their respective origins/destinations
flag=0
for i in range(0,len(energy[0])-1):
    if "Energy Stream:" in energy.loc[i,0]:
        energy_stream=energy.loc[i,0].split(',')[0].split(': ')[1]
        flag=1
    if ((flag==1) & ("FEED TO" in energy.loc[i,0])):
        if energy.loc[i+1,0].split(',')[0] != "": 
            energy_to=energy.loc[i+1,0].split(',')[0].split(": ")[1]
            equip_type=energy.loc[i+1,0].split(',')[0].split(": ")[0]
            if energy_to in equipamentos['Equipamento'].values:
                indice=equipamentos[equipamentos['Equipamento']==energy_to].index[0]
                if math.isnan(equipamentos.loc[indice,'Duty in']):
                    equipamentos.loc[indice,'Duty in']=energy_stream
                else:
                    equipamentos.loc[indice,'Duty out']=energy_stream
                    print('Aviso! Valor superposto.')
            else:
                equipamentos.loc[equipamentos.shape[0]+1,'Equipamento']=energy_to
                equipamentos.loc[equipamentos.shape[0],'Tipo']=equip_type
                equipamentos.loc[equipamentos.shape[0],'Duty in']=energy_stream
            
        if energy.loc[i+1,0].split(',')[1] != "":
            energy_from=energy.loc[i+1,0].split(',')[1].split(": ")[1]
            equip_type=energy.loc[i+1,0].split(',')[1].split(": ")[0]
            if energy_from in equipamentos['Equipamento'].values:
                indice=equipamentos[equipamentos['Equipamento']==energy_from].index[0]
                if math.isnan(equipamentos.loc[indice,'Duty out']):
                    equipamentos.loc[indice,'Duty out']=energy_stream
                else:
                    equipamentos.loc[indice,'Duty in']=energy_stream
                    print('Aviso! Valor superposto.2')
            else:
                equipamentos.loc[equipamentos.shape[0]+1,'Equipamento']=energy_from
                equipamentos.loc[equipamentos.shape[0],'Tipo']=equip_type
                equipamentos.loc[equipamentos.shape[0],'Duty out']=energy_stream
        flag=0
   
        
# In this simulation no equipment had more then 7 inputs or outputs. If there are more, the loop below should be reviewed to add input and output columns
    
#this loop attributes the material inputs and outputs to each equipment
flag=0
for i in range(0,len(materials[0])-1):
    if "Material Stream:" in materials.loc[i,0]:
        material_stream=materials.loc[i,0].split(',')[0].split(': ')[1]
        flag=1
        
        
    if ((flag==1) & ("FEED TO" in materials.loc[i,0])):
         
        destination = materials.loc[i+1,0].split(',')[0]
        origin = materials.loc[i+1,0].split(',')[1]
        
        if destination != "": # if it is an inlet
            material_to=destination.split(": ")[1]
            equip_type=destination.split(": ")[0]
            if material_to in equipamentos['Equipamento'].values:
                linha=equipamentos[equipamentos['Equipamento']==material_to].index[0]-1 #tive que colocar o -1 pq as linhas começam em 1
                if pd.isna(equipamentos.iloc[linha,4]): # 4 is the column number for Input 1
                    equipamentos.iloc[linha,4] =material_stream
                elif pd.isna(equipamentos.iloc[linha,5]):
                    equipamentos.iloc[linha,5] =material_stream
                elif pd.isna(equipamentos.iloc[linha,6]):
                    equipamentos.iloc[linha,6] =material_stream
                elif pd.isna(equipamentos.iloc[linha,7]):
                    equipamentos.iloc[linha,7] =material_stream
                elif pd.isna(equipamentos.iloc[linha,8]):
                    equipamentos.iloc[linha,8] =material_stream
                elif pd.isna(equipamentos.iloc[linha,9]):
                    equipamentos.iloc[linha,9] =material_stream
                elif pd.isna(equipamentos.iloc[linha,10]):
                    equipamentos.iloc[linha,10] =material_stream
                else:
                    print(i)
            else:
                equipamentos.loc[equipamentos.shape[0]+1,'Equipamento']=material_to
                equipamentos.loc[equipamentos.shape[0],'Tipo']=equip_type
                equipamentos.loc[equipamentos.shape[0],'Inlet 1']=material_stream
                
                
        if origin != "": #if it is an outlet
            material_from=origin.split(": ")[1]
            equip_type=origin.split(": ")[0]
            if material_from in equipamentos['Equipamento'].values:
                linha=equipamentos[equipamentos['Equipamento']==material_from].index[0]-1 #tive que colocar o -1 pq as colunas começam em 1
                if pd.isna(equipamentos.iloc[linha,11]): #9 is the column number for Output1
                    equipamentos.iloc[linha,11] =material_stream
                elif pd.isna(equipamentos.iloc[linha,12]):
                    equipamentos.iloc[linha,12] =material_stream
                elif pd.isna(equipamentos.iloc[linha,13]):
                    equipamentos.iloc[linha,13] =material_stream
                elif pd.isna(equipamentos.iloc[linha,14]):
                    equipamentos.iloc[linha,14] =material_stream
                elif pd.isna(equipamentos.iloc[linha,15]):
                    equipamentos.iloc[linha,15] =material_stream
                elif pd.isna(equipamentos.iloc[linha,16]):
                    equipamentos.iloc[linha,16] =material_stream
                elif pd.isna(equipamentos.iloc[linha,17]):
                    equipamentos.iloc[linha,17] =material_stream
                else:
                    print(i)
            else:
                equipamentos.loc[equipamentos.shape[0]+1,'Equipamento']=material_from
                equipamentos.loc[equipamentos.shape[0],'Tipo']=equip_type
                equipamentos.loc[equipamentos.shape[0],'Outlet 1']=material_stream
        flag=0   


        
#Creating table h0-s0  
h0s0 = pd.read_table(folder_path+h0s0_file, delim_whitespace=False,header=None)

h0s0 = h0s0[0].str.split(",", n = 10, expand = True) 

flag=0
h0=[]
s0=[]
mat_h0s0=[]
for i in range(0,len(h0s0[0])):
    if 'Material Stream' in h0s0.loc[i,0]:
        flag=2
        mat_h0s0.append(h0s0.loc[i,0].split(": ")[1])
    if ((flag>0) & ('EnthalpyRef' in h0s0.loc[i,0])):
        h0.append(h0s0.loc[i,2])
        flag=flag-1
    elif ((flag>0) & ('EntropyRef' in h0s0.loc[i,0])):
        s0.append(h0s0.loc[i,2])
        flag=flag-1
        
h0s0= pd.DataFrame.from_dict({'Corrente':mat_h0s0,'h0 (kJ/kmol)':h0,'s0 (kJ/kmolK)': s0})
h0s0.head()



#to find the values of energy streams
energy_values=pd.read_excel(folder_path+hysys_report, sheet_name='Energy Streams')
energy_values.drop(axis=1,index=0,inplace=True)
energy_values
#energy_values['GT comp1 power'].values[0] #para conseguir o valor de cada stream



#now, completing the table with all for each stream
streams_values=pd.read_excel(folder_path+streams_file, sheet_name='Output (1)',header=2)

dict_rows=np.array(dict_rows)-4 #subtracting 4, the number of rows skipped in the header
streams_values=streams_values.iloc[dict_rows]
streams_values=streams_values.transpose()
streams_values.insert(loc=0,column='Corrente',value=np.array(streams_values.index))
streams_values=streams_values.reset_index(drop=True).drop(axis=0,index=[0,1,2,3])
streams_values=streams_values.reset_index(drop=True)
streams_values.columns=dict_keys
for i in range(0,len(streams_values)):
    streams_values.loc[i,'Corrente']=str(streams_values.loc[i,'Corrente']) #converts streams names to strings

#merges tables so all streams have all their properties in the same table
propriedades=streams_values.merge(h0s0, on='Corrente')
propriedades.head() 

,Corrente,Fração de vapor,Temperatura (C),Pressão (kPa),Vazão molar (kmol/s),Vazão mássica (kg/s),Massa específica (kg/m³),Entalpia molar (kJ/kmol),Entropia molar (kJ/kmolK),Comp. C1,...,Comp. CO2,Comp. N2,Comp. O2,Comp. H2O,Comp. Ar,Comp. TEG,Comp. H2S,LHV (kJ/kmol),h0 (kJ/kmol),s0 (kJ/kmolK)
0,1,1,15,7000,0.916621,16.5194,64.5434,-83628.5,146.936,0.889822,...,0.019996,0.00889822,0,0,0,0,0.00019996,837774,-8.166e+004,187.4
1,2,1,117,4580,0.916621,16.5194,26.489,-78560.8,165.252,0.889822,...,0.019996,0.00889822,0,0,0,0,0.00019996,837774,-8.166e+004,187.4
2,3,1,30,200,0.00962826,0.277778,2.29152,130.203,146.503,0,...,0,0.79,0.21,0,0,0,0,0,-8.183,151.7
3,4,1,15,4670,1.9733,63.3806,65.445,-742.635,111.804,0,...,0,0.01,0.97,0,0.02,0,0,0,-9.519,145.8
4,5,1,200,4580,1.9733,63.3806,37.2808,5070.5,127.552,0,...,0,0.01,0.97,0,0.02,0,0,0,-9.519,145.8


### Exergy analysis

In [7]:
#this cell calculates the physical, chemical and total exergy, based on the references stated earlier, and appends these values to the properties by stream table

propriedades.replace('<empty>',0,inplace=True) 
propriedades.replace('',0,inplace=True)
propriedades.iloc[:,1:24]=propriedades.iloc[:,1:24].astype('float') #converting columns to floats

propriedades['Exergia física (kJ/kmol)']=propriedades['Entalpia molar (kJ/kmol)']-propriedades['h0 (kJ/kmol)']-\
T0*(propriedades['Entropia molar (kJ/kmolK)']-propriedades['s0 (kJ/kmolK)']) #cálculo da exergia física b = h-h0 - T0*(s-s0)


componentes=['Comp. C1','Comp. C2','Comp. C3','Comp. C4','Comp. C5','Comp. N2','Comp. O2','Comp. H2O','Comp. Ar','Comp. CO2','Comp. TEG','Comp. H2S']

propriedades['Exergia química (kJ/kmol)']=propriedades['Exergia física (kJ/kmol)'] #cria coluna para não ter problema de indexing depois
b_ch=[]
for i in range(0,len(propriedades)):
    b_acum=0
    for componente in propriedades[componentes].columns:
        x_comp=propriedades.loc[i,componente]
        b_std=comp_amb[comp_amb['Substância']==(componente.split('. ')[1])]['Standard chemical exergy (kJ/kmol)'].values[0]
        #preciso acertar as referências para uma referência comum nessa coluna, e aí o valor sendo importado aqui será correto
        
        if x_comp!=0:
            b_acum=b_acum+x_comp*(b_std+T0*R*np.log(x_comp))
        else:
            b_acum=b_acum+0
        #print(i,componente, x_comp, b_std, b_acum)
    propriedades.loc[i,'Exergia química (kJ/kmol)']=b_acum

    
    
propriedades['Exergia total (kJ/kmol)']=propriedades['Exergia física (kJ/kmol)']+propriedades['Exergia química (kJ/kmol)']
propriedades_uteis=['Corrente','Entalpia molar (kJ/kmol)','h0 (kJ/kmol)','Entropia molar (kJ/kmolK)','s0 (kJ/kmolK)','Exergia física (kJ/kmol)','Exergia química (kJ/kmol)','Exergia total (kJ/kmol)']

propriedades.head()

,Corrente,Fração de vapor,Temperatura (C),Pressão (kPa),Vazão molar (kmol/s),Vazão mássica (kg/s),Massa específica (kg/m³),Entalpia molar (kJ/kmol),Entropia molar (kJ/kmolK),Comp. C1,...,Comp. H2O,Comp. Ar,Comp. TEG,Comp. H2S,LHV (kJ/kmol),h0 (kJ/kmol),s0 (kJ/kmolK),Exergia física (kJ/kmol),Exergia química (kJ/kmol),Exergia total (kJ/kmol)
0,1,1.0,15.0,7000.0,0.916621,16.519444,64.543396,-83628.456200,146.936213,0.889822,...,0.0,0.00,0.0,0.0002,837774.065187,-81660.000,187.4,10095.821985,873838.120601,883933.942586
1,2,1.0,117.0,4580.0,0.916621,16.519444,26.488976,-78560.815594,165.251805,0.889822,...,0.0,0.00,0.0,0.0002,837774.065187,-81660.000,187.4,9702.668874,873838.120601,883540.789475
2,3,1.0,30.0,200.0,0.009628,0.277778,2.291515,130.202891,146.502512,0.000000,...,0.0,0.00,0.0,0.0000,0.000000,-8.183,151.7,1688.017042,128.494388,1816.511430
3,4,1.0,15.0,4670.0,1.973299,63.380556,65.445043,-742.634812,111.804199,0.000000,...,0.0,0.02,0.0,0.0000,0.000000,-9.519,145.8,9402.732339,3710.564436,13113.296774
4,5,1.0,200.0,4580.0,1.973299,63.380556,37.280758,5070.497084,127.551886,0.000000,...,0.0,0.02,0.0,0.0000,0.000000,-9.519,145.8,10520.691136,3710.564436,14231.255572


In [8]:
work_equip=['Compressor','Expander','Pump']
heat_equip=['Absorber','Conversion Reactor','Distillation','Heater','Separator'] #cooler is not listed here because it is the only equipment type where
#the heat sign convention is reversed (heat in is negative)
#Also, in my model I tried to standardize Coolers as the equipment used when cooling water is employed, which makes it easier to estimate cooling water flow.

equipamentos.replace(np.nan,0,inplace=True)
Ti=293.15 #as a standard, I considered heat being exchanged at the environment temperature T0=293.15, since the heat integrations are performed with heat
#exchangers. There are very few heaters, and for them, the heat exchange temperature is estimated in the loop.  For the coolers and other heat duties,
# they are generally rejected to the environment at T0
equipamentos['Balanço energia']=equipamentos['Equipamento'] #creating columns to assure no indexing issues
equipamentos['Balanço energia %']=equipamentos['Equipamento']
equipamentos['Exergia destruída (kW)']=equipamentos['Equipamento']
equipamentos['Eficiência exergética I']=equipamentos['Equipamento']

for i in range(1,len(equipamentos)+1):
    Q_in=0
    W_in=0
    Q_out=0
    W_out=0
    Ti=293.15
    if equipamentos.loc[i,'Tipo'] in work_equip:
        if equipamentos.loc[i,'Duty in']!=0:
            W_in=-energy_values[equipamentos.loc[i,'Duty in']].values[0] #convention for heat and work signals is already applied here
        elif equipamentos.loc[i,'Duty out']!=0:
            W_out=energy_values[equipamentos.loc[i,'Duty out']].values[0]
    elif equipamentos.loc[i,'Tipo'] in heat_equip:       
        if equipamentos.loc[i,'Duty in']!=0:
            Q_in=energy_values[equipamentos.loc[i,'Duty in']].values[0]
        elif equipamentos.loc[i,'Duty out']!=0:
            Q_out=-energy_values[equipamentos.loc[i,'Duty out']].values[0]
            
        if equipamentos.loc[i,'Tipo']=='Heater':
        #defines the heat exchange temperature as 10 C above the largest temperature found for Heaters (or 10 C below the lowest temperature, in case it is
        # a "Heater" that is in fact rejecting heat (Q<0)
            if Q_in > 0:
                Ti = max([propriedades[propriedades['Corrente']==equipamentos.loc[i,'Inlet 1']]['Temperatura (C)'].values[0],\
                propriedades[propriedades['Corrente']==equipamentos.loc[i,'Outlet 1']]['Temperatura (C)'].values[0]]) + 10 +273.15
            else:
                Ti = min([propriedades[propriedades['Corrente']==equipamentos.loc[i,'Inlet 1']]['Temperatura (C)'].values[0],\
                propriedades[propriedades['Corrente']==equipamentos.loc[i,'Outlet 1']]['Temperatura (C)'].values[0]]) - 10 +273.15
    
    elif equipamentos.loc[i,'Tipo']=='Cooler':
        Q_out=-energy_values[equipamentos.loc[i,'Duty out']].values[0]
        if Q_in > 0:
            Ti = max([propriedades[propriedades['Corrente']==equipamentos.loc[i,'Inlet 1']]['Temperatura (C)'].values[0],\
            propriedades[propriedades['Corrente']==equipamentos.loc[i,'Outlet 1']]['Temperatura (C)'].values[0]]) + 10 +273.15
            print(equipamentos.loc[i,'Equipamento'],Ti)
        else:
            Ti = 293.15
    
    else:
        equipamentos.loc[i,'Balanço energia']= 'N/A'
        
    H_in=0
    B_in=0
    H_out=0
    B_out=0
    for j in range(4,10):
        if equipamentos.iloc[i-1,j]!=0:
            n=propriedades[propriedades['Corrente']==equipamentos.iloc[i-1,j]]['Vazão molar (kmol/s)'].values[0]
            h=propriedades[propriedades['Corrente']==equipamentos.iloc[i-1,j]]['Entalpia molar (kJ/kmol)'].values[0]
            b=propriedades[propriedades['Corrente']==equipamentos.iloc[i-1,j]]['Exergia total (kJ/kmol)'].values[0]
            H_in=H_in+n*h
            B_in=B_in+n*b
            #print(equipamentos.iloc[i-1,0],equipamentos.iloc[i-1,j],B_in)
    for j in range(11,17):
        if equipamentos.iloc[i-1,j]!=0:
            n=propriedades[propriedades['Corrente']==equipamentos.iloc[i-1,j]]['Vazão molar (kmol/s)'].values[0]
            h=propriedades[propriedades['Corrente']==equipamentos.iloc[i-1,j]]['Entalpia molar (kJ/kmol)'].values[0]
            b=propriedades[propriedades['Corrente']==equipamentos.iloc[i-1,j]]['Exergia total (kJ/kmol)'].values[0]
            H_out=H_out+n*h
            B_out=B_out+n*b
            #print(equipamentos.iloc[i-1,0],equipamentos.iloc[i-1,j],B_out)

    #calculation of some indicators for equipments
    equipamentos.loc[i,'Balanço energia']= (Q_in+Q_out)-(W_in+W_out)+(H_in-H_out)
    bd=(B_in-W_in+Q_in*(1-T0/Ti))-(B_out+W_out-Q_out*(1-T0/Ti))
    equipamentos.loc[i,'Exergia destruída (kW)']= bd
    equipamentos.loc[i,'Eficiência exergética I']= 1- bd/(B_in-W_in+Q_in*(1-T0/Ti))
    #print(equipamentos.iloc[i-1,0],Q_in,Q_out,W_in,W_out,B_in,B_out)
    if H_in != 0:
        equipamentos.loc[i,'Balanço energia %']= ((Q_in+Q_out)-(W_in+W_out)+(H_in-H_out))/H_in
    else:
        equipamentos.loc[i,'Balanço energia %']=np.nan

In [9]:
equipamentos[['Equipamento','Tipo','Balanço energia','Balanço energia %','Exergia destruída (kW)','Eficiência exergética I']].sort_values(by='Eficiência exergética I',ascending=False).head(60)
#Some unit operations show exergy efficiency slightly above 1 (negative exergy destruction)
#There processes have been narrowed down, essentially, to separators. This is because of the way how HYSYS calculates the separation process considering constant
#entropy, and possibly rounding errors cause this kind of behavior. The values are very small, so these processes can be considered isentropic, and do not
#generate innacuracies in the analysis.


#equipamentos[equipamentos['Tipo']=='Heat Exchanger']

,Equipamento,Tipo,Balanço energia,Balanço energia %,Exergia destruída (kW),Eficiência exergética I
11,Indirect contact cooler,Separator,-4.36557e-11,6.96681e-18,-3479.87,1.01132
63,Separator 2,Separator,-5.82077e-11,1.44879e-16,-249.678,1.00952
39,E-109,Cooler,-1.15506e-10,2.5834e-16,-63.9404,1.00199
65,separator 3,Separator,0,-0,-41.9378,1.00147
82,V-102,Separator,5.82077e-11,-1.27847e-16,-45.1459,1.00132
42,NG gas heater,Heat Exchanger,-4645.11,0.0605971,-442.358,1.00055
77,V-100,Separator,0,-0,-13.6906,1.00046
76,E-103,Heat Exchanger,-458.139,0.000251978,-6.86701,1.00009
81,TEE-107,Tee,-0.00042112,1.09417e-09,-0.00042112,1
45,HP cooling split,Tee,0,-0,-7.567e-10,1


In [10]:
propriedades.set_index('Corrente',inplace=True)
comp_amb.set_index('Substância',inplace=True)

In [11]:
subsistemas=pd.DataFrame(columns=['Subsistema','Bd','Balanço energia']) #making indexes available
equipamentos.set_index('Equipamento',inplace=True)

### Defining the function the calculates indicators for subsystems. The functions takes a list of material inlets, a list of material outlets, and a list of 
#energy streams. These lists have to be input by the users based on the control volume frontier that they would like to analyze.
def calc_subsist(inlets,outlets,energy):
    H_in=0
    H_out=0
    B_in=0
    B_out=0
    W_in=0
    W_out=0
    Q_net=0
    B_q=0
    m_in=0
    m_out=0
    
    for stream in inlets:
        vazao=propriedades.loc[stream,'Vazão molar (kmol/s)']
        m_in=m_in+vazao
        H_in=H_in+vazao*propriedades.loc[stream,'Entalpia molar (kJ/kmol)']
        B_in=B_in+propriedades.loc[stream,'Exergia total (kJ/kmol)']*vazao
            
    for stream in outlets:
        vazao=propriedades.loc[stream,'Vazão molar (kmol/s)']
        m_out=m_out+vazao
        H_out=H_out+propriedades.loc[stream,'Entalpia molar (kJ/kmol)']*vazao
        B_out=B_out+propriedades.loc[stream,'Exergia total (kJ/kmol)']*vazao

    for stream in energy:
        if stream in equipamentos[['Duty in']].values:
            tipo = equipamentos[equipamentos['Duty in']==stream]['Tipo'].values[0]
            equip=equipamentos[equipamentos['Duty in']==stream].index[0]
        elif stream in equipamentos[['Duty out']].values:
            tipo = equipamentos[equipamentos['Duty out']==stream]['Tipo'].values[0]
            equip=equipamentos[equipamentos['Duty out']==stream].index[0]
        else:
            print('Error! Energy stream not found.')
        
        valor=energy_values[stream].values[0]
        
        if tipo=='Expander':
            W_out=W_out+valor
        elif ((tipo=='Compressor') | (tipo=='Pump')):
            W_in=W_in-valor
        elif tipo in heat_equip:
            Tj=T0
            Q_net=Q_net+valor
            B_q=B_q+valor*(1-T0/Tj)
        elif tipo=='Cooler':
            Tj=T0
            Q_net=Q_net-valor
            B_q=B_q-valor*(1-T0/Tj)
            
             
    Bd = (B_in-B_out)-(W_in+W_out)+B_q
    balanco_energia = Q_net - (W_in+W_out) + H_in-H_out 
    balanco_material = m_in-m_out                           
                               
    variaveis=[H_in,H_out,B_in,B_out,W_in,W_out,Q_net,B_q,m_in,m_out,Bd,balanco_energia,balanco_material]                          
    colunas=['H_in','H_out','B_in','B_out','W_in','W_out','Q_net','B_q','m_in','m_out','Bd','Balanço energia','Balanço material']
    resultado=pd.DataFrame(columns=colunas)
    for i in range(0,len(variaveis)):
        resultado.loc[0,colunas[i]]=variaveis[i]
    return resultado

#### Calculating some subsystems that are useful for indicators

In [12]:
#oxyturbine-cycle

In [13]:
#ASU

#this subsystem is handled as a fixed energy cost per weight of purified O2

In [14]:
#CO2pur-comp

inlets=['to dryers','TEG in']
outlets=['TEG to recycle','pur-10','pur-23']
Win=['Q-110','Q-111','Q-113','Q-115']
Wout=['Q-114']
Q=['Q-107','dist_cond_duty','Q-112']
energy=Win+Wout+Q
results_pur= calc_subsist(inlets,outlets,energy)
results_pur

,H_in,H_out,B_in,B_out,W_in,W_out,Q_net,B_q,m_in,m_out,Bd,Balanço energia,Balanço material
0,-415197,-421316,62327.4,62963.3,-10979.8,636.157,-16534.7,0,1.09644,1.09627,9707.79,-71.9238,0.000166247


In [15]:
#utility-offsite

equipamentos_energy=equipamentos[['Tipo','Duty in','Duty out']]
for row in equipamentos_energy.index:
    if equipamentos_energy.loc[row,'Duty in'] in list(energy_values.columns.values):
        equipamentos_energy.loc[row,'Duty in'] = energy_values.loc[1,(equipamentos_energy.loc[row,'Duty in'])]
    if equipamentos_energy.loc[row,'Duty out'] in energy_values.columns.values:
        equipamentos_energy.loc[row,'Duty out'] = energy_values.loc[1,(equipamentos_energy.loc[row,'Duty out'])]
        
        
a=equipamentos_energy[['Tipo','Duty in','Duty out']].groupby(by='Tipo').sum()
b=a.loc['Conversion Reactor','Duty in']-a.loc['Cooler','Duty out']+a.loc['Separator','Duty in']+a.loc['Heater','Duty in']+40000
b=-400000 #This was estimated as the heat rejection rate to cooling water by inspection of the model. I still need to find a better way of calculating this.
#I tried standardizing Coolers as heat rejection to cooling water, but some key components such as condensers were left out, as they are "Separator" equipment
#Also, for example, HRSG heat loss is a heat rejection to the environment air, and not to cooling water, but falls under the "cooler" category
m_cw=-b/(10*4.182)
m_cw
W_cw=(1/1000)*m_cw*200
W_cw #cooling water compression power in kW

#this cell calculates the cooling water flow and duty based on heat rejected from the system (as of this version, input by the user in variable b)

C:\Users\Italo\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Italo\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Italo\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1912.9603060736486

In [16]:
#gas-turbine

inlets=['1','5','6']
outlets=['10']
Win=['GT comp1 power','GT comp2 power']
Wout=['GT1 power','GT2 power','GT3 power','GT4 power']
Q=['CC heat loss']
energy=Win+Wout+Q
results_gt= calc_subsist(inlets,outlets,energy)
results_gt

,H_in,H_out,B_in,B_out,W_in,W_out,Q_net,B_q,m_in,m_out,Bd,Balanço energia,Balanço material
0,-5.51654e+06,-5.80917e+06,1.10801e+06,533907,-261236,540947,-17271.1,0,17.9258,17.9689,294396,-4349.09,-0.0431133


In [17]:
#steam-turbine

inlets=['10']
outlets=['11']
Win=['HP pump power','MP pump power','Cond pump power']
Wout=['LP ST power','MP ST power','HP ST power']
Q=['HRSG heat loss','Q-100']
energy=Win+Wout+Q
results_st= calc_subsist(inlets,outlets,energy)
results_st

,H_in,H_out,B_in,B_out,W_in,W_out,Q_net,B_q,m_in,m_out,Bd,Balanço energia,Balanço material
0,-5.80917e+06,-6.2399e+06,533907,310136,-2266.61,169105,-27879.1,0,17.9689,17.9689,56933,236009,0


In [18]:
#Complete cycle
inlets_ciclo=['1','4-','TEG in']
outlets_ciclo=['pur-23','pur-10','TEG to recycle','14*']
Win_ciclo=['GT comp1 power','GT comp2 power','O2 comp power','Cond pump power','FG comp1 power','FG comp2 power','HP pump power','MP pump power',\
    'Q-110','Q-111','Q-113']
Wout_ciclo=['GT1 power','GT2 power','GT3 power','GT4 power','HP ST power','MP ST power','LP ST power','Q-114']
Q_ciclo=['Q-107','Q-112','dist_cond_duty','FG comp IC1','FG comp IC2','HRSG heat loss','ICC cooling','Condenser cooling',\
   'Q-100','Q-101','CC heat loss']
energy_ciclo=Win_ciclo+Wout_ciclo+Q_ciclo
result_ciclo = calc_subsist(inlets_ciclo,outlets_ciclo,energy_ciclo)

result_ciclo


,H_in,H_out,B_in,B_out,W_in,W_out,Q_net,B_q,m_in,m_out,Bd,Balanço energia,Balanço material
0,-99430.4,-954565,851326,84826.5,-329325,710687,-479072,0,2.9177,2.96064,385137,-5299.82,-0.042947


In [19]:
#this cell arranges the results in a table for comparison between different power cycles
#It was written considering certain "given conditions" such as that the power plant being analyzed is an oxy-fuel gas turbine plant, such that the oxidizer
#inlet stream is O2 rich, the fuel inlet stream contains methane, and that there is a high purity CO2 outlet stream destinated to CCS

ciclo_results=pd.DataFrame(columns=['Variável','Unidade','Valor']).set_index('Variável')

for stream in inlets_ciclo:
    if propriedades.loc[stream,'Comp. C1']!=0:
        fuel=stream
    elif propriedades.loc[stream,'Comp. O2']>0.8:
        oxy=stream

for stream in outlets_ciclo:
    if propriedades.loc[stream,'Comp. CO2']>0.85:
        ccs=stream

ciclo_results.loc['Fuel mass flow','Valor'] = propriedades.loc[fuel,'Vazão mássica (kg/s)']*3.6 #convertendo para ton/h
ciclo_results.loc['Fuel mass flow','Unidade'] ='t/h'        
        
ciclo_results.loc['Fuel LHV','Valor'] = propriedades.loc[fuel,'LHV (kJ/kmol)']
ciclo_results.loc['Fuel LHV','Unidade'] ='kJ/kmol'

ciclo_results.loc['Thermal energy input','Valor'] = propriedades.loc[fuel,'LHV (kJ/kmol)']*propriedades.loc[fuel,'Vazão molar (kmol/s)']/1000
ciclo_results.loc['Thermal energy input','Unidade'] ='MWt'

ciclo_results.loc['Gas turbine output','Valor'] = (results_gt['W_out'].values[0]+results_gt['W_in'].values[0])*efic_mec/1000
ciclo_results.loc['Gas turbine output','Unidade'] ='MWe'

ciclo_results.loc['Steam turbine output','Valor'] = (results_st['W_out'].values[0]+results_st['W_in'].values[0])*efic_mec/1000
ciclo_results.loc['Steam turbine output','Unidade'] ='MWe'

ciclo_results.loc['Gross electric output','Valor'] = ciclo_results.loc['Gas turbine output','Valor']+ciclo_results.loc['Steam turbine output','Valor']
ciclo_results.loc['Gross electric output','Unidade'] ='MWe'

#ciclo_results.loc['Gross electric output','Valor'] = result_ciclo['W_out'].values[0]*efic_mec/1000
#ciclo_results.loc['Gross electric output','Unidade'] ='MWe'

ciclo_results.loc['Oxyturbine cycle','Valor'] = 16410/efic_mec/1000 #energy_values['O2 comp power'].values[0]*efic_mec/1000
ciclo_results.loc['Oxyturbine cycle','Unidade'] ='MWe'

ciclo_results.loc['Air separation unit','Valor'] = propriedades.loc[oxy,'Vazão mássica (kg/s)']*oxy_cost/efic_mec/1000
ciclo_results.loc['Air separation unit','Unidade'] ='MWe'

ciclo_results.loc['CO2 purification and compression power','Valor'] = -(results_pur['W_in'].values[0]+results_pur['W_out'].values[0])/efic_mec/1000
ciclo_results.loc['CO2 purification and compression power','Unidade'] ='MWe'

ciclo_results.loc['Utility and offsite units','Valor'] = W_cw/efic_mec/1000 
ciclo_results.loc['Utility and offsite units','Unidade'] ='MWe'

ciclo_results.loc['Electric power consumption','Valor'] = ciclo_results.loc['Oxyturbine cycle','Valor']+ciclo_results.loc['Air separation unit','Valor']\
+ciclo_results.loc['CO2 purification and compression power','Valor']+ciclo_results.loc['Utility and offsite units','Valor']
ciclo_results.loc['Electric power consumption','Unidade'] ='MWe'

#esse código abaixo é o consumo total do sistema "ciclo_completo"
#ciclo_results.loc['Electric power consumption','Valor'] = (-result_ciclo['W_in'].values[0]\
                                                           #+propriedades.loc[oxy,'Vazão mássica (kg/s)']*oxy_cost)*efic_mec/1000
#ciclo_results.loc['Electric power consumption','Unidade'] ='MWe'

ciclo_results.loc['Net electric output','Valor'] = (ciclo_results.loc['Gross electric output','Valor']\
- ciclo_results.loc['Electric power consumption','Valor'])*efic_transf
ciclo_results.loc['Net electric output','Unidade'] ='MWe'

ciclo_results.loc['Gross electrical efficiency','Valor'] = ciclo_results.loc['Gross electric output','Valor']*100\
/ciclo_results.loc['Thermal energy input','Valor']
ciclo_results.loc['Gross electrical efficiency','Unidade'] ='%'

ciclo_results.loc['Net electrical efficiency','Valor'] = ciclo_results.loc['Net electric output','Valor']*100\
/ciclo_results.loc['Thermal energy input','Valor']
ciclo_results.loc['Net electrical efficiency','Unidade'] ='%'

ciclo_results.loc['Equivalent CO2 in fuel','Valor'] = propriedades.loc[fuel,'Vazão molar (kmol/s)']*\
(propriedades.loc[fuel,'Comp. C1']+2*propriedades.loc[fuel,'Comp. C2']+3*propriedades.loc[fuel,'Comp. C3']+\
 4*propriedades.loc[fuel,'Comp. C4']+5*propriedades.loc[fuel,'Comp. C5'])*3600
ciclo_results.loc['Equivalent CO2 in fuel','Unidade'] ='kmol/h'

ciclo_results.loc['Captured CO2','Valor'] = propriedades.loc[ccs,'Vazão molar (kmol/s)']*\
propriedades.loc[ccs,'Comp. CO2']*3600
ciclo_results.loc['Captured CO2','Unidade'] ='kmol/h'

ciclo_results.loc['CO2 removal efficiency','Valor'] = 100*ciclo_results.loc['Captured CO2','Valor']/ciclo_results.loc['Equivalent CO2 in fuel','Valor']
ciclo_results.loc['CO2 removal efficiency','Unidade'] ='%'

ciclo_results.loc['Fuel consumption per net power','Valor'] = ciclo_results.loc['Thermal energy input','Valor']/ciclo_results.loc['Net electric output','Valor']
ciclo_results.loc['Fuel consumption per net power','Unidade'] ='MWt/MWe'

ciclo_results.loc['CO2 emission per net power','Valor'] = (ciclo_results.loc['Equivalent CO2 in fuel','Valor']\
    -ciclo_results.loc['Captured CO2','Valor'])*comp_amb.loc['CO2','Massa molar (kg/kmol)']/ciclo_results.loc['Net electric output','Valor']
ciclo_results.loc['CO2 emission per net power','Unidade'] ='kg/MWh'

ciclo_results

,Unidade,Valor
Variável,,
Fuel mass flow,t/h,59.47
Fuel LHV,kJ/kmol,837774
Thermal energy input,MWt,767.922
Gas turbine output,MWe,278.312
Steam turbine output,MWe,166.004
Gross electric output,MWe,444.316
Oxyturbine cycle,MWe,16.4925
Air separation unit,MWe,65.3552
CO2 purification and compression power,MWe,10.3957
